In [1]:
#core data lib
import pandas as pd
import pandasql as ps

In [2]:
#https://ourworldindata.org/
#https://github.com/owid/covid-19-data/tree/master/public/data/
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

In [3]:
#df.head(10) #continent

In [4]:
df = df.sort_values(by=['date'])
df = df.reset_index()

In [66]:
df2 = ps.sqldf('''select date, continent, location, new_cases from df where date>='2022-01-01' and total_cases>0 and location='Russia' order by date, new_cases''')

In [67]:
#NaN to 0
df2['new_cases'] = df2['new_cases'].fillna(0)

#datetime to int(Epoch)
df2['date'] = pd.to_datetime(df2['date'])
df2['date'] = (pd.to_numeric(df2['date'])/1000000000).astype(int)

#text to string
df2['continent'] = df2['continent'].astype(str)
df2['location'] = df2['location'].astype(str)


,date,continent,location,new_cases
0,1640995200,Europe,Russia,19606.0
1,1641081600,Europe,Russia,18088.0
2,1641168000,Europe,Russia,16193.0
3,1641254400,Europe,Russia,15760.0
4,1641340800,Europe,Russia,15632.0


In [93]:
df2.tail(5)

,date,continent,location,new_cases
249,1662508800,Europe,Russia,47354.0
250,1662595200,Europe,Russia,49884.0
251,1662681600,Europe,Russia,51420.0
252,1662768000,Europe,Russia,52708.0
253,1662854400,Europe,Russia,52155.0


In [68]:
#import catboost as cb
Y_train = df2['new_cases']#.head(10000)
X_train = df2.loc[:, df2.columns !='new_cases']#.head(10000)

cat_features = [1, 2]

In [69]:
X_train
#X_train['continent']

,date,continent,location
0,1640995200,Europe,Russia
1,1641081600,Europe,Russia
2,1641168000,Europe,Russia
3,1641254400,Europe,Russia
4,1641340800,Europe,Russia
...,...,...,...
249,1662508800,Europe,Russia
250,1662595200,Europe,Russia
251,1662681600,Europe,Russia
252,1662768000,Europe,Russia


In [70]:
# Initialize data
from catboost import Pool
train_dataset = Pool(X_train, Y_train, cat_features)

cat_features = [1, 2]

In [71]:
#limits
limit_iterations = 10000
limit_metric_period = int(limit_iterations/10)

limit_metric_period

1000

In [72]:
#import catboost
from catboost import CatBoostRegressor

# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=limit_iterations,
                          learning_rate=1,
                          depth=3)
# Fit model
model.fit(train_dataset, metric_period=limit_metric_period) #, plot='true'


0:	learn: 27484.5571946	total: 1.64ms	remaining: 16.4s
1000:	learn: 27.1534122	total: 222ms	remaining: 1.99s
2000:	learn: 2.9879401	total: 416ms	remaining: 1.66s
3000:	learn: 0.3571084	total: 611ms	remaining: 1.43s
4000:	learn: 0.0414798	total: 805ms	remaining: 1.21s
5000:	learn: 0.0049395	total: 999ms	remaining: 999ms
6000:	learn: 0.0006021	total: 1.19s	remaining: 795ms
7000:	learn: 0.0000703	total: 1.39s	remaining: 594ms
8000:	learn: 0.0000087	total: 1.58s	remaining: 395ms
9000:	learn: 0.0000011	total: 1.77s	remaining: 197ms
9999:	learn: 0.0000001	total: 1.97s	remaining: 0us


In [121]:
pedict_df = df2.loc[:, df2.columns !='new_cases'].tail(1)#+86400
pedict_df['date'] = pedict_df['date']+86400
pedict_df.reset_index(drop=True, inplace=True)
pedict_df

,date,continent,location
0,1662940800,Europe,Russia


In [128]:
# Get predictions
preds = model.predict(pedict_df)
#preds
pd.DataFrame(preds)#.columns = ['new_cases']

,0
0,52155.0


In [131]:
new_row_df = pedict_df
new_row_df = new_row_df.join(pd.DataFrame(preds))
new_row_df.rename(columns={0: 'new_cases'}, inplace=True)
new_row_df

,date,continent,location,new_cases
0,1662940800,Europe,Russia,52155.0
